# LOGİSTİC REGRESSİON

İNSTAGRAM:cuneytaykac_

Faydalı olması dileklerimle...
# #Evde Kal

Mavi takımın kazanma ve kaybetme sayıları üzerinde logistic regression yapıcaz yani modelime ögreticem.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

BlueWins =1 kazandı,BlueWins=0 ise kaybetti


gameld ile işim olmadıgı için drop ettim.

In [ ]:
data.drop(["gameId"],axis=1,inplace=True)

In [ ]:
data.head()

x ve y degerlerimizi belirledim

In [ ]:
y=data.blueWins.values
x_data=data.drop(["blueWins"],axis=1)

x_data degerlerimi normalize edicem çünkü plot ederken bazı degerlerin yok  olmaması için

In [ ]:
x=(x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

Sıra geldi elimizde datayı bölme işlemine.Elimizdeki data yı test ve train olarak bölmemiz lazım ben %25 test %75 train şeklinde bölücem.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

##### random_state=42 yazmamızın nedeni plot ettiricegimiz zaman random şekilde degerleri random şekilde bölme işlemi yapıcak ve her zaman aynı random degerinde bölme işlemi gerçekleştiricek.

In [ ]:
x_train = x_train.T #columlarla rowların yerini degiştirdik x ler için
x_test = x_test.T
y_train = y_train.T#rowlarla columların yerini degiştirdik y ler için
y_test = y_test.T

In [ ]:
print("x_train: ",x_train.shape)
print("x_test: ",x_test.shape)
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)


In [ ]:
#parameter initialize
#dimension:38
def initialize_weights_and_bias(dimension):
    w=np.full((dimension,1),0.01)
    b=0.0
    return w,b



sigmoid function matematiksel ifadesi y=1/1+e üzeri -x



In [ ]:
#sigmoid function
def sigmoid(z):
    y_head=1/(1+np.exp(-z))
    return y_head

## FOGWARD AND BACKWARD PROPAGATİONS

* w:weight
* b: bias
* x:x_train
* T:Transpoz alma


* Aşagıdaki kodun açıklaması
## forward propagation: İlk olarak X degerlerimizi weight'lerle çarpıp bias ile toplarız.Sonuc olarak bir deger çıkar bu degere z deriz. bu z degerimizi Sigmoid Function'ın içine yollarız.Sigmoid function bu z degerimizi 0 ile bir degeri  arasında bir sonuç çıktısı verir.Bu sonuca Y_head degeri deriz.


## Elde ettigimiz bu y_head degeri ile asıl olan y degerimizi karşılaştırırız.hata varmı yokmu diye.Bu hata bulma olayına loss fonction denir.
## matematiksel ifadesi 
* (-1-y) * log(1-y_head) -y*log(1-y_head)

## Bu denklemi tüm y degerleri için uygularız ve cost degerlerimizi buluruz,ve toplarız.


In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    #fogward propagation
    z=np.dot(w.T,x_train)+b
    y_head=sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost=(np.sum(loss))/x_train.shape[1]
    #backward_propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] 
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost,gradients

Backward Propagation ise türev alma işlemidir.Weight ve bias degerlerimin türevlerini almaktır.

# Update



* Learning_rate :Ögrenme hızı 
* Number of iteration:Deneme sayısı
* w:w-slope
* b:b-slope


In [ ]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
         # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
            
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

# Predict (Tahmin etme)

Bu kısımda tahmin etme olayını gerçekleştiricem.Sigmoid fucntion da bir TreShoul degeri belirledik bu deger 0.5.
* Eger z degerim 0.5 den büyükse y_head=1
* Eger z degerim 0.5 den kucukse y_head=0 , tahmin edicek.

In [ ]:
def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    # if z is bigger than 0.5, our prediction is sign one (y_head=1),
    # if z is smaller than 0.5, our prediction is sign zero (y_head=0),
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

# Logistic Regression

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    dimension =  x_train.shape[0]  
    w,b = initialize_weights_and_bias(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
     
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)

    # Print test Errors
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    
logistic_regression(x_train, y_train, x_test, y_test,learning_rate =1 , num_iterations = 250)

Umarım faydalı olmuştur.Begenmeden geçmeyelim :)